# Importing Libraries

In [1]:
import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
from mpl_toolkits.basemap import Basemap
from sklearn import metrics, linear_model
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from scipy.optimize import curve_fit
from scipy import stats
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'scipy'

In [ ]:
conda install -c conda-forge basemap-data-hires

Read the flights and airlines dataset.

In [ ]:
flights = pd.read_csv('../input/flight-delays/flights.csv')

In [ ]:
airports = pd.read_csv("../input/flight-delays/airports.csv")

# Overview of all Flights

In [ ]:
count_flights = flights['ORIGIN_AIRPORT'].value_counts()
#___________________________
plt.figure(figsize=(9,9))
#________________________________________
# Define the variables with their values
colors = ['gray', 'red', 'darkblue', 'violet', 'green', 'orange']
size_limits = [1, 100, 1000, 10000, 100000, 1000000]
labels = []
for i in range(len(size_limits)-1):
    labels.append("{} <.< {}".format(size_limits[i], size_limits[i+1]))
    
# To create the base outline of the map visualization
map = Basemap(resolution='i',llcrnrlon=-180, urcrnrlon=-50,
              llcrnrlat=10, urcrnrlat=75, lat_0=0, lon_0=0,)
map.drawcoastlines()
map.shadedrelief()
map.drawcountries(linewidth = 1)
map.drawstates(color='0.1')

# Put airports on map
for index, (code, y,x) in airports[['IATA_CODE', 'LATITUDE', 'LONGITUDE']].iterrows():
    x, y = map(x, y)
    isize = [i for i, val in enumerate(size_limits) if val < count_flights[code]]
    ind = isize[-1]
    map.plot(x, y, marker='o', markersize = ind+5, markeredgewidth = 1, color = colors[ind],
             markeredgecolor='k', label = labels[ind])

# Remove duplicate labels and set their order
handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
key_order = ('1 <.< 100', '100 <.< 1000', '1000 <.< 10000',
             '10000 <.< 100000', '100000 <.< 1000000')
new_label = OrderedDict()
for key in key_order:
    new_label[key] = by_label[key]
plt.legend(new_label.values(), new_label.keys(), loc = 1, prop= {'size':9},
           title='Number of flights per year', frameon = True, framealpha = 1)
plt.show()

Bring records of only January due to the large size of the dataset. This is done by calling the '1' value in the dataset which depicts the month January.

In [ ]:
flights = flights[flights['MONTH'] == 1]

# Data Transformation

Converting the existing dates into YYYY-MM-DD format for easier reading.

In [ ]:
flights['DATE'] = pd.to_datetime(flights[['YEAR','MONTH', 'DAY']])
print(flights['DATE'])

The following functions help convert the confusing time format into an understanding format.

In [ ]:
# Convert the HHMM format of scheduled departure into a datetime format
def format_hours(time):
    if pd.isnull(time):
        return np.nan
    else:
        if time == 2400: time = 0
        time = "{0:04d}".format(int(time))
        hour = datetime.time(int(time[0:2]), int(time[2:4]))
        return hour
#_____________________________________________________________________
# Function that combines a date and time to produce a datetime.datetime
def combine_date_hour(x):
    if pd.isnull(x[0]) or pd.isnull(x[1]):
        return np.nan
    else:
        return datetime.datetime.combine(x[0],x[1])
#_______________________________________________________________________________
# Function that combine two columns of the dataframe to create a datetime format in one column
def create_flight_time(flights, col):    
    liste = []
    for index, cols in flights[['DATE', col]].iterrows():    
        if pd.isnull(cols[1]):
            liste.append(np.nan)
        elif float(cols[1]) == 2400:
            cols[0] += datetime.timedelta(days=1)
            cols[1] = datetime.time(0,0)
            liste.append(combine_date_hour(cols))
        else:
            cols[1] = format_hours(cols[1])
            liste.append(combine_date_hour(cols))
    return pd.Series(liste)

In [ ]:
# Calling the premade functions to modify the existing data
flights['SCHEDULED_DEPARTURE'] = create_flight_time(flights, 'SCHEDULED_DEPARTURE')
flights['DEPARTURE_TIME'] = flights['DEPARTURE_TIME'].apply(format_hours)
flights['SCHEDULED_ARRIVAL'] = flights['SCHEDULED_ARRIVAL'].apply(format_hours)
flights['ARRIVAL_TIME'] = flights['ARRIVAL_TIME'].apply(format_hours)
# Retrieve the modified columns with the changed data
flights.loc[:10, ['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL', 'DEPARTURE_TIME','ARRIVAL_TIME', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']]

Through analyzing the result, it can be shown that the variables, **DEPARTURE_TIME** and **ARRIVAL_TIME** do not show any clear understanding of whether the flight was delayed or not. In the dataset, it is not depicted in a understandable manner. It does not indicate if there is a large delay or if the flight left before time. Therefore, the variables will not be used. Another set of variables will be used to show if the flight was delayed or not.

# Data Filling

In [ ]:
variables_to_remove = ['TAXI_OUT', 'TAXI_IN', 'WHEELS_ON', 'WHEELS_OFF', 'YEAR', 
                       'MONTH','DAY','DAY_OF_WEEK','DATE', 'AIR_SYSTEM_DELAY',
                       'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
                       'WEATHER_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
                       'FLIGHT_NUMBER', 'TAIL_NUMBER', 'AIR_TIME']
# Drop the columns completely and modify the data in place
flights.drop(variables_to_remove, axis = 1)
flights = flights[['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
        'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY',
        'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY',
        'SCHEDULED_TIME', 'ELAPSED_TIME']]
flights[:5]

Finding the missing values and removing the entries completely. There is no viable option to assume the data for the blanks in the dataset.

In [ ]:
# Find the missing values of each row in the dataset
missing_flights = flights.isnull().sum(axis=0).reset_index()
# Create the columns to show the variable and the number of missing values
missing_flights.columns = ['variable', 'missing values']
# Calculate the % of whether to fill those fields or remove
missing_flights['filling factor (%)']=(flights.shape[0]-missing_flights['missing values'])/flights.shape[0]*100
# Drops the old index range and makes it in increasing order
missing_flights.sort_values('filling factor (%)').reset_index(drop = True)

In [ ]:
# Drop the missing values directly
flights.dropna()

# Basic Statistics

In [ ]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
#_______________________________________________________________
# Creation of a dataframe with statitical infos on each airline:
global_stats = flights['DEPARTURE_DELAY'].groupby(flights['AIRLINE']).apply(get_stats).unstack()
global_stats = global_stats.sort_values('count')
global_stats

# Comparing Airlines - Which has the most delay?

Reading the airlines.csv file for accessing the airline names and respective IATA codes.

In [ ]:
airlines_names = pd.read_csv('../input/flight-delays/airlines.csv')
airlines_names

In [ ]:
iata_airlines = airlines_names.set_index('IATA_CODE')['AIRLINE'].to_dict()

To check which airline has the most or least delay, we would have to create a graphic visualization to compare each airline. This visualization compares on time, short and long delays for each US airline.

In [ ]:
#_____________________________________________
# Function that define how delays are grouped
delay_type = lambda x:((0,1)[x > 5],2)[x > 45]
flights['DELAY_LEVEL'] = flights['DEPARTURE_DELAY'].apply(delay_type)
#____________________________________________________
fig = plt.figure(1, figsize=(10,7))
ax = sns.countplot(y="AIRLINE", hue='DELAY_LEVEL', data=flights)
#____________________________________________________________________________________
# We replace the abbreviations by the full names of the companies and set the labels
labels = [iata_airlines[item.get_text()] for item in ax.get_yticklabels()]
ax.set_yticklabels(labels)
plt.setp(ax.get_xticklabels(), fontsize=10, weight = 'normal', rotation = 0);
plt.setp(ax.get_yticklabels(), fontsize=10, weight = 'bold', rotation = 0);
ax.yaxis.label.set_visible(False)
plt.xlabel('Flight Count', fontsize=16, labelpad=10)
#________________
# Set the legend
L = plt.legend()
L.get_texts()[0].set_text('on time (t < 5 min)')
L.get_texts()[1].set_text('small delay (5 < t < 45 min)')
L.get_texts()[2].set_text('large delay (t > 45 min)')
plt.show()

# Compare Delay in Take off or Arrival

The visualization shows that the departures are higher than the arrivals. It can be safe to assume that the average delays between the airlines is much higher during take off rather than arrival at the destination. Another assumption for this could be flights adjusting their flight speed during the landing phase which in turn can play a big role in the arrival delay.

In [ ]:
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['hatch.linewidth'] = 2.0  

fig = plt.figure(1, figsize=(5,5))
ax = sns.barplot(x="DEPARTURE_DELAY", y="AIRLINE", data=flights, color="grey", ci=None)
ax = sns.barplot(x="ARRIVAL_DELAY", y="AIRLINE", data=flights, hatch = '///',
                 alpha = 0.0, ci=None)
labels = [iata_airlines[item.get_text()] for item in ax.get_yticklabels()]
ax.set_yticklabels(labels)
ax.yaxis.label.set_visible(False)
plt.xlabel('Average delay between the departures, @grey and arrivals, hatch lines',
           fontsize=10, labelpad=10);

# Prediction of Flight Delays

The goal of this model is to predict the flight delay time for the month January 2021(the last week) during the COVID period. We would categorise this delay along with small or large delays.

In [ ]:
df_train = flights[flights['SCHEDULED_DEPARTURE'].apply(lambda x:x.date()) < datetime.date(2015, 1, 23)]
df_test  = flights[flights['SCHEDULED_DEPARTURE'].apply(lambda x:x.date()) > datetime.date(2015, 1, 23)]
df = df_train

# Linear Regression

**LINEAR REGRESSION INTRODUCTION FOR ALGORITHM.**

This model will take into account all the airports in comparison with a single airline. All of the flights will be from the month of January. The specific airline which will be considered is American Airlines.

In [ ]:
def get_flight_delays(flights, carrier, id_airport, extreme_values = False):
    df2 = flights[(flights['AIRLINE'] == carrier) & (flights['ORIGIN_AIRPORT'] == id_airport)]
    #_______________________________________
    # remove extreme values before fitting
    if extreme_values:
        df2['DEPARTURE_DELAY'] = df2['DEPARTURE_DELAY'].apply(lambda x:x if x < 60 else np.nan)
        df2.dropna(how = 'any')
    #__________________________________
    # Converting date and hour to solely hour
    df2.sort_values('SCHEDULED_DEPARTURE', inplace = True)
    df2['hour_depart'] =  df2['SCHEDULED_DEPARTURE'].apply(lambda x:x.time())
    #___________________________________________________________________
    test2 = df2['DEPARTURE_DELAY'].groupby(df2['hour_depart']).apply(get_stats).unstack()
    test2.reset_index(inplace=True)
    #___________________________________
    fct = lambda x:x.hour*3600+x.minute*60+x.second
    test2.reset_index(inplace=True)
    test2['hour_depart_min'] = test2['hour_depart'].apply(fct)
    return test2

In [ ]:
def merged_delays(flights, carrier):
    listed_airports = flights[flights['AIRLINE'] == carrier]['ORIGIN_AIRPORT'].unique()
    i = 0
    listed_columns = ['AIRPORT_ID', 'hour_depart_min', 'mean']
    for id_airport in listed_airports:
        test2 = get_flight_delays(flights, carrier, id_airport, True)
        test2.loc[:, 'AIRPORT_ID'] = id_airport
        test2 = test2[listed_columns]
        test2.dropna(how = 'any', inplace = True)
        if i == 0:
            merged_df = test2.copy()
        else:
            merged_df = pd.concat([merged_df, test2], ignore_index = True)
        i += 1    
    return merged_df

Filter out only AA flights.

In [ ]:
carrier = 'AA'
merged_df = merged_delays(flights, carrier)
merged_df.shape

In [ ]:
# Assigning different labels for each airport in the airports.csv file
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(merged_df['AIRPORT_ID'])
#__________________________________________________________
# Similarity between the codes and tags of the airports
zipped = zip(integer_encoded, merged_df['AIRPORT_ID'])
label_airports = list(set(list(zipped)))
label_airports.sort(key = lambda x:x[0])
label_airports[:5]

**ONE HOT ENCODING EXPLANATION HERE.**

Throughout the use of the one hot encoding method, we create an array which replaces the current airport labels into the previous labels which were set for the airports which is in numerical format.

In [ ]:
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
b = np.array(merged_df['hour_depart_min'])
b = b.reshape(len(b),1)
X = np.hstack((onehot_encoded, b))
Y = np.array(merged_df['mean'])
Y = Y.reshape(len(Y), 1)
print(X.shape, Y.shape)

**MSE SCORE INTRODUCTION**

In [ ]:
lm = linear_model.LinearRegression()
model = lm.fit(X,Y)
predictions = lm.predict(X)
print("MSE =", round(metrics.mean_squared_error(predictions, Y)))

In [ ]:
tips = pd.DataFrame()
tips["Airports"] = pd.Series([float(s) for s in predictions]) 
tips["Mean_delays"] = pd.Series([float(s) for s in Y]) 
sns.jointplot(x="Mean_delays", y="Airports", data=tips, size = 4, ratio = 5,
              joint_kws={'line_kws':{'color':'red'}}, kind='reg')
plt.show()
print("Pearsonr:", stats.pearsonr(tips["Airports"], tips["Mean_delays"]))

**Pearsonr score introduction**

The model shows a positive correlation between the airport predictions along with the delays. The current MSE score is 54 which is not that great of a score as it is not well accurate with the real data. A solution would be to try a polynomial regression fit.

# Polynomial Regression

**INTRODUCTION HERE.**

In [ ]:
poly = PolynomialFeatures(degree = 2)
regr = linear_model.LinearRegression()
X_ = poly.fit_transform(X)
regr.fit(X_, Y)

In [ ]:
result = regr.predict(X_)
print("MSE =", round(metrics.mean_squared_error(result, Y)))

In [ ]:
tips = pd.DataFrame()
tips["Airports"] = pd.Series([float(s) for s in result]) 
tips["Mean_delays"] = pd.Series([float(s) for s in Y]) 
sns.jointplot(x="Mean_delays", y="Airports", data=tips, size = 4, ratio = 7,
              joint_kws={'line_kws':{'color':'red'}}, kind='reg')
plt.show()
print("Pearsonr:", stats.pearsonr(tips["Airports"], tips["Mean_delays"]))

There is a definite difference between the two models in terms of MSE and Pearson r score. However, there is still a high positive correlation.

# Splitting The Dataset

This process is splitting the dataset into training and test data which can help reduce overfitting and biased results. Hence, this will provide a better view of the predictions. Regularization technique will be performed.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
X_train.shape

In [ ]:
poly = PolynomialFeatures(degree = 2)
regr = linear_model.LinearRegression()
X_ = poly.fit_transform(X_train)
regr.fit(X_, Y_train)
result = regr.predict(X_)
score = metrics.mean_squared_error(result, Y_train)
print("Mean squared error = ", score)

In [ ]:
X_ = poly.fit_transform(X_test)
result = regr.predict(X_)
score = metrics.mean_squared_error(result, Y_test)
print("Mean squared error = ", score)

**Ridge Regularization explanation here**

In [ ]:
ridgereg = Ridge(alpha=0.3,normalize=True)
poly = PolynomialFeatures(degree = 2)
X_ = poly.fit_transform(X_train)
ridgereg.fit(X_, Y_train)

In [ ]:
X_ = poly.fit_transform(X_test)
result = ridgereg.predict(X_)
score = metrics.mean_squared_error(result, Y_test)
print("Mean squared error = ", score)

# Testing Model

Testing the predictions for the month of January end.

In [ ]:
carrier = 'AA'
merged_df_test = merged_delays(df_test, carrier)

In [ ]:
X_ = poly.fit_transform(X_test)
result = ridgereg.predict(X_)
score = metrics.mean_squared_error(result, Y_test)
'MSE = {:.2f}'.format(score)

In [ ]:
'Average Delay = {:.2f} min'.format(np.sqrt(score))